# **Human-in-the-Loop en AutoGen**

El enfoque **Human-in-the-Loop** (HITL) en AutoGen permite la interacción directa entre un usuario humano y un equipo de agentes en tiempo real. Esto es útil para: ✅ **Dar feedback en tiempo real** mientras el equipo está en ejecución.  
✅ **Modificar la conversación antes de continuar** en una nueva ejecución.  
✅ **Combinar IA y supervisión humana** para mejorar respuestas.

---

## **1️⃣ Métodos para Proporcionar Feedback**

Existen **dos formas principales** de interactuar con un equipo de agentes y proporcionar feedback:

|**Método**|**Descripción**|**Uso recomendado**|
|---|---|---|
|**Durante la ejecución**|Se usa un `UserProxyAgent` para que el usuario pueda intervenir en tiempo real.|Para interacciones **inmediatas** (aprobaciones, selecciones rápidas).|
|**Después de la ejecución**|Se proporciona feedback en la siguiente llamada a `run()`.|Para ajustes en una **sesión persistente**.|

---

## **2️⃣ Proporcionar Feedback Durante una Ejecución**

Se usa el **`UserProxyAgent`**, un agente especial que actúa como **intermediario entre el usuario y el equipo**.

📌 **¿Cómo funciona?**

1. El equipo ejecuta su tarea.
2. En cierto momento, **se detiene y espera la intervención del usuario**.
3. Una vez el usuario proporciona feedback, la ejecución continúa.

💡 **Ejemplo: Usar `UserProxyAgent` en un equipo de RoundRobinGroupChat**

In [ ]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
import os
# Crear los agentes
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", api_key=os.environ["OPENAI_API_KEY"])
assistant = AssistantAgent("assistant", model_client=model_client)

# Agente que permite intervención humana a través de input()
user_proxy = UserProxyAgent("user_proxy", input_func=input)

# Condición de terminación: el equipo se detendrá si el usuario dice "APPROVE"
termination = TextMentionTermination("APPROVE")

# Crear el equipo con el asistente y el usuario proxy
team = RoundRobinGroupChat([assistant, user_proxy], termination_condition=termination)

# Ejecutar el equipo en streaming
await Console(team.run_stream(task="Write a 4-line poem about the ocean."))
